In [37]:
!python -V
!pip install tensorflow==2.12.0

Python 3.10.11
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
!nvidia-smi

Tue May  2 12:34:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    28W /  70W |    103MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [39]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True) 

Mounted at /content/gdrive


In [40]:
!git clone https://github.com/elw1s/MSNET.git
!cp -r "/content/MSNET/msnet/examples/msnet" .
!cp -r "/content/MSNET/demo" "msnet"
!mkdir "/content/MSNET/msnet/examples/msnet/data_dir"
!mkdir "/content/MSNET/msnet/examples/msnet/checkpoint_dir"

fatal: destination path 'MSNET' already exists and is not an empty directory.
mkdir: cannot create directory ‘/content/MSNET/msnet/examples/msnet/data_dir’: File exists
mkdir: cannot create directory ‘/content/MSNET/msnet/examples/msnet/checkpoint_dir’: File exists


In [41]:
import requests
from tqdm import tqdm

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id , 'confirm': 1 }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in tqdm(response.iter_content(CHUNK_SIZE)):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [42]:
# Get dataset
DATASET_GDRIVE_ID = "1kEKJ8kr1aScXz_1El7Mn-Yi0ANducQIW"
DATASET_PATH = "/content/MSNET/msnet/examples/msnet/data_dir/data.zip"

# Get model files
MODEL_INDEX_GDRIVE_ID = "1G2JkD1BgYxWN40NsIvCOZiet4AEPbRBf"
MODEL_INDEX_PATH = "/content/MSNET/msnet/examples/msnet/checkpoint_dir/model-7400.index"

MODEL_WEIGHTS_GDRIVE_ID = "1MK8CNs_ZXhFhX6NYDSek4Y9NGHwMaOOb"
MODEL_WEIGHTS_PATH = "/content/MSNET/msnet/examples/msnet/checkpoint_dir/model-7400.data-00000-of-00001"

In [ ]:
download_file_from_google_drive(MODEL_INDEX_GDRIVE_ID, MODEL_INDEX_PATH)
download_file_from_google_drive(MODEL_WEIGHTS_GDRIVE_ID, MODEL_WEIGHTS_PATH)
download_file_from_google_drive(DATASET_GDRIVE_ID, DATASET_PATH)

2it [00:00, 378.60it/s]
16278it [00:02, 5440.44it/s]


In [ ]:
!sudo apt install python3.10


In [ ]:
!unzip "/content/MSNET/msnet/examples/msnet/data_dir/data.zip" -d "/content/MSNET/msnet/examples/msnet/data_dir"

In [ ]:
!cp -f /content/MSNET/env/coco.py /usr/local/lib/python3.9/dist-packages/pycocotools/coco.py
!cp -f /content/MSNET/env/cocoeval.py /usr/local/lib/python3.9/dist-packages/pycocotools/cocoeval.py

##Dependencies

In [ ]:
!pip install --upgrade pip


In [ ]:
#Cuda
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

!lspci | grep -i nvidia
### gcc compiler is required for development using the cuda toolkit. to verify the version of gcc install enter
!gcc --version
# system update
!sudo apt-get update
!sudo apt-get upgrade
# install other import packages
!sudo apt-get install g++ freeglut3-dev build-essential libx11-dev libxmu-dev libxi-dev libglu1-mesa libglu1-mesa-dev
# first get the PPA repository driver
!sudo add-apt-repository ppa:graphics-drivers/ppa
!sudo apt update
# install nvidia driver with dependencies
!sudo apt install libnvidia-common-470
!sudo apt install libnvidia-gl-470
!sudo apt install nvidia-driver-470

 # installing CUDA-11.8

!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-ubuntu2004.pin
!sudo mv cuda-ubuntu2004.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda-repo-ubuntu2004-11-8-local_11.8.0-520.61.05-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu2004-11-8-local_11.8.0-520.61.05-1_amd64.deb
!sudo cp /var/cuda-repo-ubuntu2004-11-8-local/cuda-*-keyring.gpg /usr/share/keyrings/
!sudo apt-get update
!sudo apt-get -y install cuda

# installing cudnn 8.6
!wget https://developer.nvidia.com/compute/cudnn/secure/8.6.0/local_installers/11.8/cudnn-local-repo-ubuntu2004-8.6.0.163_1.0-1_amd64.deb
!sudo dpkg -i cudnn-local-repo-ubuntu2004-8.6.0.163_1.0-1_amd64.deb
!sudo cp /var/cudnn-repo-ubuntu2004-8-6-local/cudnn-*-keyring.gpg /usr/share/keyrings/
!sudo apt-get update
!sudo apt-get -y install cuda


In [ ]:
!nvidia-smi


In [ ]:
!nvcc -V


In [ ]:
import pycocotools.mask as cocomask
import sys

org_paths = sys.path
sys.path[2] = '/usr/lib/python310.zip'
sys.path[3] = '/usr/lib/python3.10'
sys.path[4] = '/usr/lib/python3.10/lib-dynload'
sys.path[6] = '/usr/local/lib/python3.10/dist-packages'
sys.path[8] = '/usr/local/lib/python3.10/dist-packages/IPython/extensions'

In [ ]:
!pip install tensorpack

In [ ]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')

print(tf.__version__)
print("GPU:", tf.config.list_physical_devices('GPU'))
print("Num GPUs:", len(physical_devices))
tf.test.gpu_device_name()

In [ ]:
!python3.10 /content/MSNET/msnet/examples/msnet/train.py --config DATA.BASEDIR=/content/MSNET/msnet/examples/msnet/data_dir MODE_FPN=True \
  TRAIN.BASE_LR=1e-3 TRAIN.EVAL_PERIOD=1 TRAIN.LR_SCHEDULE=[3000]  \
  PREPROC.TRAIN_SHORT_EDGE_SIZE=[600,1200] TRAIN.CHECKPOINT_PERIOD=1 DATA.NUM_WORKERS=1 \
  --load /content/MSNET/msnet/examples/msnet/checkpoint_dir \
  --logdir /content/MSNET/msnet/examples/msnet/log_dir
